# Improve agentic reasoning

* in-context learning
    * one-shot, few-shots learning (examples of using tools and/or reasoning)
    * LLM as pattern completion/following machines

* reasoning, up-front planning, and chain of thought
    * build a plan before doing anything, step-by-step
        * go and run the plan
            * turn into code, workflow language
            * repeatability (save and re-run the code)
        * put the plan into memory
            * execute actions one at the time
            * flexible reaction on errors
        * planning as a tool
            * generate the plan using expensive hi-quality model
            * saves on execution 

### Extending the Agent Loop with Capabilities

The **Capability** pattern lets you extend core agent behavior without modifying the agent loop. Each capability encapsulates a focused adaptation (e.g., logging, time-awareness, metrics), plugs into lifecycle hooks, and cleanly composes with others. This keeps the loop minimal while enabling rich behaviors like opening DB connections, logging LLM prompts, or injecting metadata.

#### Interaction Points in the Agent Loop

```python
def run(self, user_input: str, memory=None, action_context_props=None):
    # ... existing code ...

    # Initialize capabilities
    for capability in self.capabilities:
        capability.init(self, action_context)
        
    while True:
        # Start-of-loop
        can_start_loop = reduce(lambda a, c: c.start_agent_loop(self, action_context),
                               self.capabilities, False)

        # Build prompt
        prompt = reduce(lambda p, c: c.process_prompt(self, action_context, p),
                        self.capabilities, base_prompt)

        # Process LLM response
        response = reduce(lambda r, c: c.process_response(self, action_context, r),
                          self.capabilities, response)

        # Process parsed action
        action = reduce(lambda a, c: c.process_action(self, action_context, a),
                        self.capabilities, action)

        # Execute & post-process result
        result = reduce(lambda r, c: c.process_result(self, action_context, response,
                                                      action_def, action, r),
                        self.capabilities, result)

        # End-of-loop
        for capability in self.capabilities:
            capability.end_agent_loop(self, action_context)
```

#### Capability Interface

```python
class Capability:
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description

    def init(self, agent, action_context: ActionContext) -> dict: ...
    def start_agent_loop(self, agent, action_context: ActionContext) -> bool: return True
    def process_prompt(self, agent, action_context: ActionContext, prompt: Prompt) -> Prompt: return prompt
    def process_response(self, agent, action_context: ActionContext, response: str) -> str: return response
    def process_action(self, agent, action_context: ActionContext, action: dict) -> dict: return action
    def process_result(self, agent, action_context: ActionContext, response: str, action_def: Action,
                       action: dict, result: any) -> any: return result
    def process_new_memories(self, agent, action_context: ActionContext, memory: Memory,
                             response, result, memories: List[dict]) -> List[dict]: return memories
    def end_agent_loop(self, agent, action_context: ActionContext): ...
    def should_terminate(self, agent, action_context: ActionContext, response: str) -> bool: return False
    def terminate(self, agent, action_context: ActionContext) -> dict: ...
```

**Lifecycle cheat sheet**

* **init**: one-time setup (e.g., seed memory, open connections).
* **start_agent_loop**: pre-iteration checks (e.g., rate limits).
* **process_prompt**: mutate/augment prompts pre-LLM (e.g., add time).
* **process_response**: validate/sanitize raw LLM text.
* **process_action**: enrich/validate planned actions.
* **process_result**: annotate/transform tool results.
* **process_new_memories**: redact/augment memories before storing.
* **end_agent_loop**: cleanup/logging per iteration.
* **should_terminate / terminate**: graceful shutdown & finalization.

#### Composing Capabilities into an Agent

```python
class Agent:
    def __init__(self,
                 goals: List[Goal],
                 agent_language: AgentLanguage,
                 action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str],
                 environment: Environment,
                 capabilities: List[Capability] = [],
                 max_iterations: int = 10,
                 max_duration_seconds: int = 180):
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment
        self.capabilities = capabilities or []
        self.max_iterations = max_iterations
        self.max_duration_seconds = max_duration_seconds
```

```python
agent = Agent(
    goals=[Goal(name="scheduling", description="Schedule meetings considering current time and availability")],
    agent_language=JSONAgentLanguage(),
    action_registry=registry,
    generate_response=llm.generate,
    environment=PythonEnvironment(),
    capabilities=[
        TimeAwareCapability(),
        LoggingCapability(log_level="INFO"),
        MetricsCapability(metrics_server="prometheus:9090"),
    ]
)
```

*Order matters*: each capability sees and may modify the artifact before it reaches the next capability (middleware-style).

---

### Implementing Time Awareness

Make the agent consistently aware of current time for better scheduling, deadline handling, and time-sensitive decisions.

```python
from datetime import datetime
from zoneinfo import ZoneInfo

class TimeAwareCapability(Capability):
    def __init__(self):
        super().__init__(name="Time Awareness", description="Allows the agent to be aware of time")

    def init(self, agent, action_context: ActionContext) -> dict:
        tz_name = action_context.get("time_zone", "America/Chicago")
        now = datetime.now(ZoneInfo(tz_name))
        iso_time = now.strftime("%Y-%m-%dT%H:%M:%S%z")
        human_time = now.strftime("%H:%M %A, %B %d, %Y")

        memory = action_context.get_memory()
        memory.add_memory({
            "type": "system",
            "content": (
                f"Right now, it is {human_time} (ISO: {iso_time}). "
                f"You are in the {tz_name} timezone. "
                "Please consider the day/time, if relevant, when responding."
            )
        })

    def process_prompt(self, agent, action_context: ActionContext, prompt: Prompt) -> Prompt:
        tz_name = action_context.get("time_zone", "America/Chicago")
        now = datetime.now(ZoneInfo(tz_name))
        system_msg = f"Current time: {now.strftime('%H:%M %A, %B %d, %Y')} ({tz_name})\n\n"

        messages = prompt.messages
        if messages and messages[0]["role"] == "system":
            messages[0]["content"] = system_msg + messages[0]["content"]
        else:
            messages.insert(0, {"role": "system", "content": system_msg})
        return Prompt(messages=messages)
```

**Example behavior**

```
agent.run("Schedule a team meeting for today")
# → "Since it's already 5:30 PM on Friday, I recommend Monday morning. Should I check availability for Monday?"
```

---

### Extending Time Awareness

Track execution timestamps and durations for actions and results.

```python
class EnhancedTimeAwareCapability(TimeAwareCapability):
    def process_action(self, agent, action_context: ActionContext, action: dict) -> dict:
        action["execution_time"] = datetime.now(
            ZoneInfo(action_context.get("time_zone", "America/Chicago"))
        ).isoformat()
        return action

    def process_result(self, agent, action_context: ActionContext, response: str,
                       action_def: Action, action: dict, result: any) -> any:
        if isinstance(result, dict) and "execution_time" in action:
            start = datetime.fromisoformat(action["execution_time"])
            end = datetime.now(ZoneInfo(action_context.get("time_zone", "America/Chicago")))
            result["action_duration"] = (end - start).total_seconds()
        return result
```

---

### Why the Capability Pattern Works

* **Separation of concerns**: core loop stays small; features live in focused classes.
* **Composability**: mix-and-match behaviors (time, logging, metrics, safety).
* **Testability**: unit-test capabilities in isolation; mock `ActionContext`.
* **Observability & governance**: capabilities can add audit logs, PII redaction, or policy checks without touching loop logic.

Use capabilities whenever a concern cuts across multiple phases of the loop (prompting, actions, results, memory). Keep each capability single-purpose, and order them intentionally to get predictable, maintainable behavior.


### Plan First: a Capability that makes the agent think before acting

Below is a clean, drop-in implementation of a **PlanFirstCapability** plus a robust `create_plan` tool. It fixes small issues (e.g., `_memory` → `memory`), keeps your agent loop untouched, and stores the generated plan in memory so later turns can reference it. Optional progress tracking is included.

#### Capability

```python
from datetime import datetime
from typing import Optional, List

class PlanFirstCapability(Capability):
    def __init__(self, plan_memory_type: str = "system", track_progress: bool = False):
        super().__init__(
            name="Plan First Capability",
            description="The Agent will always create a plan and add it to memory"
        )
        self.plan_memory_type = plan_memory_type
        self.track_progress = track_progress
        self._initialized = False

    def _get_mem_items(self, memory: "Memory") -> List[dict]:
        # compatible with either .items or .get_memories()
        return getattr(memory, "items", None) or memory.get_memories()

    def init(self, agent, action_context: ActionContext):
        if self._initialized:
            return
        self._initialized = True

        mem = action_context.get_memory()
        action_registry = action_context.get("action_registry") or action_context.get_action_registry()

        plan_text = create_plan(
            action_context=action_context,
            memory=mem,
            action_registry=action_registry
        )

        mem.add_memory({
            "type": self.plan_memory_type,
            "content": "You must follow these instructions carefully to complete the task:\n" + plan_text,
        })

        if self.track_progress:
            mem.add_memory({
                "type": "system",
                "content": "Planning initialized. Track progress by appending step completions to memory under type=progress."
            })

    def process_prompt(self, agent, action_context: ActionContext, prompt: "Prompt") -> "Prompt":
        """Light reminder to follow the plan at each turn."""
        mem = action_context.get_memory()
        # grab the latest stored plan (if multiple, keep the last)
        plan_msgs = [m for m in self._get_mem_items(mem) if "instructions carefully" in m.get("content", "")]
        if plan_msgs:
            plan_head = "Follow the stored plan. If a step is complete, move to the next; otherwise add needed info.\n\n"
            if prompt.messages and prompt.messages[0].get("role") == "system":
                prompt.messages[0]["content"] = plan_head + prompt.messages[0]["content"]
            else:
                prompt.messages.insert(0, {"role": "system", "content": plan_head})
        return prompt

    def process_result(self, agent, action_context: ActionContext, response: str, action_def: "Action",
                       action: dict, result: any) -> any:
        """Optionally track progress after tool execution."""
        if not self.track_progress:
            return result
        mem = action_context.get_memory()
        step_name = (action or {}).get("tool") or (action_def.name if hasattr(action_def, "name") else "unknown_action")
        mem.add_memory({
            "type": "progress",
            "content": f"Completed step via `{step_name}` at {datetime.utcnow().isoformat()}Z"
        })
        return result
```

#### Planning tool

```python
@register_tool(tags=["planning"], description="Create a detailed execution plan based on the task and available tools")
def create_plan(action_context: ActionContext,
                memory: "Memory",
                action_registry: "ActionRegistry") -> str:
    """Create a detailed execution plan based on the task and available tools."""

    # Gather tool descriptions for the LLM
    try:
        actions = action_registry.get_actions()
    except AttributeError:
        actions = getattr(action_registry, "actions", [])  # fallback
    tool_descriptions = "\n".join(
        f"- {getattr(a, 'name', getattr(a, 'tool', 'tool'))}: {getattr(a, 'description', '')}"
        for a in actions
    ) or "- (no tools registered)"

    # Pull relevant memory (user/system) to ground the plan
    mem_items = getattr(memory, "items", None) or memory.get_memories()
    memory_content = "\n".join(
        f"{m.get('type', 'unknown')}: {m.get('content', '')}"
        for m in mem_items
        if m.get("type") in ("user", "system")
    ) or "(no prior task context)"

    prompt = f"""Given the task in memory and the available tools, create a detailed plan.
Think through this step by step:

1. Identify the key components of the task
2. Consider what tools you have available
3. Break down the task into logical steps
4. For each step, specify:
   - What needs to be done
   - What tool(s) will be used
   - What information is needed
   - What the expected outcome is

Write your plan in clear, numbered steps. Each step should be specific and actionable.

Available tools:
{tool_descriptions}

Task context from memory:
{memory_content}

Create a plan that accomplishes this task effectively."""
    # Call the LLM via context
    llm = action_context.get("llm")
    if not llm:
        raise ValueError("No LLM found in ActionContext under key 'llm'")
    return llm(prompt)
```

#### Wiring (what the agent needs)

Make sure your `ActionContext` provides the LLM, memory, and the action registry so the capability can generate its plan:

```python
agent = Agent(
    goals=[Goal(name="analysis", description="Analyze sales data and create a report")],
    agent_language=JSONAgentLanguage(),
    action_registry=registry,
    generate_response=llm.generate,
    environment=PythonEnvironment(),
    capabilities=[PlanFirstCapability(track_progress=True)]
)

# When you run:
# - Your agent loop should build an ActionContext that includes:
#   {"memory": memory, "llm": llm.generate, "action_registry": registry, ...}
result = agent.run("Analyze our Q4 sales data and create a report")
```

#### Notes & best practices

* **Separation of concerns**: the capability composes planning into the loop; the loop stays unchanged.
* **Idempotence**: `init` runs once per agent run to avoid duplicate plans.
* **Robustness**: the tool works with either `memory.items` or `memory.get_memories()`.
* **Progress tracking** (optional): writes `type="progress"` memories after each action, useful for audits and UI.


## In-loop planning

* in planning ahead, the planning is not necessary the focus at the end of the execution
* another approach
    * create a plan
        * at the end of iteration, create progress report
            * where are we on the plan,
            * what was done,
            * what is next
            * can be done every iteration, of at every 5th...

### Tracking Progress: Capability + Tool

Below is a polished, drop-in implementation of a **`track_progress`** tool and a **`ProgressTrackingCapability`** that appends a concise reflection at the **end of each loop iteration**. It fixes minor issues (`_memory` → `memory`), supports dependency injection via `ActionContext`, and lets you control frequency and memory type.

#### Progress tool

```python
@register_tool(tags=["prompts"], description="Summarize progress and recommend next steps")
def track_progress(action_context: ActionContext,
                   memory: "Memory",
                   action_registry: "ActionRegistry") -> str:
    """Generate a progress report based on the current task, available tools, and memory context."""

    # 1) Tool catalog
    try:
        actions = action_registry.get_actions()
    except AttributeError:
        actions = getattr(action_registry, "actions", [])
    tool_descriptions = "\n".join(
        f"- {getattr(a, 'name', getattr(a, 'tool', 'tool'))}: {getattr(a, 'description', '')}"
        for a in actions
    ) or "- (no tools registered)"

    # 2) Relevant memory (user/system + latest progress/status/results if you keep those types)
    mem_items = getattr(memory, "items", None) or memory.get_memories()
    relevant_types = {"user", "system", "progress", "status", "result"}
    memory_content = "\n".join(
        f"{m.get('type','unknown')}: {m.get('content','')}"
        for m in mem_items
        if m.get("type") in relevant_types
    ) or "(no prior task context)"

    # 3) Prompt
    prompt = f"""Given the current task and available tools, generate a progress report.
Think step by step:

1) Identify key components of the task and intended outcome.
2) Assess progress so far from the provided context.
3) Identify blockers or issues.
4) Recommend concrete next steps.
5) Suggest any tool usage to advance.

Write using short, structured bullets: Progress • Blockers • Next Steps • Suggested Tools.

Available tools:
{tool_descriptions}

Task context from memory:
{memory_content}

Return only the report text."""
    llm = action_context.get("llm")
    if not llm:
        raise ValueError("No LLM found in ActionContext under key 'llm'")
    return llm(prompt)
```

#### Capability

```python
from datetime import datetime

class ProgressTrackingCapability(Capability):
    def __init__(self,
                 memory_type: str = "system",
                 track_frequency: int = 1,
                 max_report_chars: int = 2000,
                 tag: str = "Progress Report"):
        """
        memory_type: where to store reports (e.g., 'system'/'progress')
        track_frequency: run every N iterations (1 = every loop)
        max_report_chars: truncate very long reports to bound token cost
        tag: label inserted into memory for easier retrieval
        """
        super().__init__(
            name="Progress Tracking",
            description="Tracks progress and enables reflection after actions"
        )
        self.memory_type = memory_type
        self.track_frequency = max(1, int(track_frequency))
        self.max_report_chars = max(200, int(max_report_chars))
        self.tag = tag
        self.iteration_count = 0

    def end_agent_loop(self, agent, action_context: ActionContext):
        """Generate and store a progress report at the end of selected iterations."""
        self.iteration_count += 1
        if self.iteration_count % self.track_frequency != 0:
            return

        mem = action_context.get_memory()
        # You can inject either via context or a helper:
        action_registry = (action_context.get("action_registry")
                           or action_context.get_action_registry())

        try:
            report = track_progress(
                action_context=action_context,
                memory=mem,
                action_registry=action_registry
            )
        except Exception as e:
            # Log into memory for observability without crashing the loop
            mem.add_memory({
                "type": "system",
                "content": f"[{self.tag}] Error generating progress report: {e}"
            })
            return

        if not isinstance(report, str):
            report = str(report)

        # Hard cap to keep costs predictable
        if len(report) > self.max_report_chars:
            report = report[: self.max_report_chars] + " …[truncated]"

        mem.add_memory({
            "type": self.memory_type,
            "content": f"{self.tag} (Iteration {self.iteration_count}, UTC {datetime.utcnow().isoformat()}Z):\n{report}"
        })

    # Optional: surface termination hint if repeated blockers detected
    def should_terminate(self, agent, action_context: ActionContext, response: str) -> bool:
        """Example heuristic: if the last N reports mention 'blocked' repeatedly, suggest stop."""
        # Keep it disabled by default; implement if you maintain blocker counters in memory.
        return False
```

#### Wiring into your agent

```python
agent = Agent(
    goals=[Goal(name="data_processing", description="Process and analyze customer feedback data")],
    agent_language=JSONAgentLanguage(),
    action_registry=registry,
    generate_response=llm.generate,
    environment=PythonEnvironment(),
    capabilities=[
        ProgressTrackingCapability(
            memory_type="progress",     # or "system"
            track_frequency=2,          # reflect every 2nd iteration
            max_report_chars=1500,
            tag="Progress Report"
        )
    ]
)

# Ensure your ActionContext includes:
# {"memory": memory, "llm": llm.generate, "action_registry": registry, ...}
result = agent.run("Analyze customer feedback from Q4 and identify top issues")
```

#### Practical tips

* **Cost control:** Use `track_frequency`, `max_report_chars`, and concise output instructions to bound tokens.
* **Signal over noise:** Store reports under a dedicated `type` (e.g., `"progress"`) so downstream steps can quickly fetch recent reflections without scanning all memory.
* **Close the loop:** When planning the next action, have your planner/selective-context step include the **latest progress report** to avoid repeating completed work and to address blockers promptly.
* **Security:** If memory may contain sensitive data, add a redaction capability before storing reports (e.g., mask PII).

This pattern adds a lightweight “**act → reflect → adjust**” rhythm to the agent, improving robustness on long, multi-tool workflows without polluting the core loop.


## Ahead of time vs dynamic

* execution styles
    * agent decides every-step what to do next
        * adaptive
        * flexible
        * expensive
        * slow
        * unpredictable
        * error prone
    * up-front planning, code or workflow generated
        * no control once the plan is generated
        * rigid
        * fast
        * predictable
        * cheap
    * hybrid
        * up-front planning
        * on error/problem generate adaptive response
    * shim
        * agent validating inputs to the next step